Main goal of this notebook is to use the google earth engine to identify and create features that represent the shoreline/waterline of the wetland projects based on different parameters like NDWI, VV and VH backscatter, and individual mutispectral bands

In [ ]:
## before anything you need to visit the site below and make sure you have a google earth engine account
## this is so you can access Sentinel-1 GRD and Sentinel-2 TOA and SR products, as well as other sensor packages and data types

## visit the below website below to setup an earth engine account, enable a cloud project, and enable the ee API 
## https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup#get-access-to-earth-engine

In [1]:
import ee
import geemap
import geemap.colormaps as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
## only need to run this once
## after authenticating with google earth engine you will only need to initialize each session

## https://developers.google.com/earth-engine/guides/auth
ee.Authenticate()

In [ ]:
## init ee cloud project you made during initial setup
ee.Initialize(project = '') ##enter your project name here as a string to initialize exchanges with ee api

In [ ]:
# cm.plot_colormaps(width=12, height=0.4)

# Some functions for a bit easier mapping
super simple for now, might make them better later

In [ ]:
## Function to add RGB images to the map.
def add_rgb_to_map(image, map_object, coll):

    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
    for band in coll.first().bandNames().getInfo(): ## all images if small enough image collection
        map_object.addLayer(image, {'min': 0, 'max': 2000, 'bands': ['B4', 'B3', 'B2']}, f'{date}_rgb')

## Function to add spectral indices images to the map.
def add_ind_to_map(image, map_object, band):

    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
    if band =='NDWI':
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.ndvi}, f'{date}_{band}')
    elif band =='NDVI': 
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.ndwi}, f'{date}_{band}')
    elif band == 'MSAVI2':
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.RdYlGn}, f'{date}_{band}')


## Function to add spectral indices images to the map.
def add_sar_to_map(image, map_object, target_band):

    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
    map_object.addLayer(image, {'min': -50, 'max': 1, 'bands': target_band}, f'{date}_{target_band}')

In [ ]:
## functiont to create three important and popular spectral indices
## ndvi = Normalized Difference Vegetation Index, good for vegetation health and cover
## ndwi = Normalized Difference Water Index, good for identifying water bodies and mositure in surface
def s2_10m_target_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    msavi2 = image.expression(
        '((2 * NIR + 1) - ((2 * NIR + 1) ** 2 - 8 * (NIR - RED)) ** 0.5) / 2',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }
    ).rename('MSAVI2')
    return image.addBands([ndvi, ndwi, msavi2])

## collects sentinel-1 GRD (radar, no phase) and Sentinel-2 SR (multispectral, adjusted for top of atmosphere reflectance)
def get_sentinel_imagery(aoi, start_date, end_date, s2_cloud_cov, orbit):
    ## Sentinel-1 ImageCollection
    s1_VV = (ee.ImageCollection('COPERNICUS/S1_GRD')
               .filterBounds(aoi)
               .filterDate(ee.Date(start_date), ee.Date(end_date))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .filter(ee.Filter.eq('orbitProperties_pass', orbit))
               .select(['VV'])
               .sort('date')
    )

    s1_VH = (ee.ImageCollection('COPERNICUS/S1_GRD')
               .filterBounds(aoi)
               .filterDate(ee.Date(start_date), ee.Date(end_date))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .filter(ee.Filter.eq('orbitProperties_pass', orbit))
               .select(['VH'])
               .sort('date')
    )

    ## Clip all images in the collection to the AOI
    s1_VV = s1_VV.map(lambda img: img.clip(aoi))
    s1_VH = s1_VH.map(lambda img:img.clip(aoi))
    ## Sentinel-2 Surface Reflectance Harmonized ImageCollection
    s2_10m = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
               .filterBounds(aoi)
               .filterDate(ee.Date(start_date), ee.Date(end_date))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', s2_cloud_cov))
               .sort('date')
               .select(['B2', 'B3', 'B4', 'B8'])
    )
    ## Clip all images in the collection to the AOI
    s2_10m = s2_10m.map(lambda img: img.clip(aoi))
    ## Apply indices to the Sentinel-2 images
    s2_10m_ndvi = s2_10m.map(s2_10m_target_indices).select(['NDVI'])
    s2_10m_ndwi = s2_10m.map(s2_10m_target_indices).select(['NDWI'])
    s2_10m_msavi2 = s2_10m.map(s2_10m_target_indices).select(['MSAVI2'])

    
    return s1_VV, s1_VH, s2_10m, s2_10m_ndvi, s2_10m_ndwi, s2_10m_msavi2


In [ ]:
## fucntion to get the date of each image in the image collection
def get_date(image):
    return ee.Feature(None, {'date': image.date().format('YYYY-MM-dd')})

In [ ]:
def export_image_to_drive(collection, collname, image, index):
    # Define the description for the export, incorporating the index for uniqueness

    if collname == 's1_VV':
        description = f"s1_VV_{index}"
    elif collname == 's1_VH':
        description = f's1_VH_{index}'
    elif collname == 'rgb':
        description = f's2_10m_{index}'
    elif collname == 'ndvi':
        description = f's2_10m_ndvi_{index}'
    elif collname == 'ndwi':
        description = f's2_10m_ndwi_{index}'
    else:
        description = f's2_10m_msavi2_{index}'

    # Setup the export task
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        region=aoi,  # Make sure the geometry is defined earlier
        fileFormat='GeoTIFF',
        scale = 10
    )
    task.start()
    print(f'Exporting {description} to Drive...')

def export_all_images(collection, collname):
    image_list = collection.toList(collection.size())  # Convert ImageCollection to List
    num_images = image_list.size().getInfo()  # Get the number of images

    if collname[:2] == 's1':
        for i, date in enumerate(s1_date_list):
            image = ee.Image(image_list.get(i))
            export_image_to_drive(collection, collname, image, date[:10])

    else:
        for i, date in enumerate(s2_date_list):
            image = ee.Image(image_list.get(i))
            export_image_to_drive(collection, collname, image, date[:10])

In [ ]:
# Define a function to get histogram of NDWI for each image
def get_histogram(image, scale, bucket_num, band_name):
    """
    Used to create ndwi histograms for the imagery

    image = ee.Image
        NDWI image to determine the shoreline from
    scale = int
        scale to estimate the histogram from, typically 10 to match the resolution of the RGB imagery
    bucket_num = int
        number of buckets to put the data into for histogram
    band_name = str
        the name of your target band in the image
    """


    # Reduce the image to get a histogram over the region of interest (ROI)
    hist = image.reduceRegion(
        reducer=ee.Reducer.histogram(maxBuckets=bucket_num),  # Adjust the number of buckets as needed
        geometry=aoi,
        scale=scale,  # Adjust based on image resolution
        maxPixels=1e8
    )
    
    # Get the histogram data for NDWI
    histogram = ee.Dictionary(hist.get(band_name)).getInfo() 
    
    return histogram

In [ ]:
def otsu_trimodal_from_histogram(histogram):
    # Extract histogram and means from the provided NDWI histogram
    counts = np.array(histogram['histogram'])
    means = np.array(histogram['bucketMeans'])
    total = np.sum(counts)
    sum_values = np.sum(means * counts)
    overall_mean = sum_values / total
    size = len(means)

    # Define a function to compute the between-class variance (BSS)
    def compute_intervariance(i, k):
        # Compute for first region (A)
        aCounts = counts[:i]
        aCount = np.sum(aCounts)
        aMeans = means[:i]
        aMean = np.sum(aMeans * aCounts) / aCount if aCount != 0 else 0

        # Compute for second region (B)
        bCounts = counts[i:k]
        bCount = np.sum(bCounts)
        bMeans = means[i:k]
        bMean = np.sum(bMeans * bCounts) / bCount if bCount != 0 else 0

        # Compute for third region (C)
        cCounts = counts[k:]
        cCount = np.sum(cCounts)
        cMeans = means[k:]
        cMean = np.sum(cMeans * cCounts) / cCount if cCount != 0 else 0

        # Return combined BSS
        return aCount * (aMean - overall_mean) ** 2 + \
               bCount * (bMean - overall_mean) ** 2 + \
               cCount * (cMean - overall_mean) ** 2

    # Initialize an empty list for BSS values
    bss_values = []
    
    # Iterate through potential thresholds
    for i in range(1, size - 1):  # Start at 1, and stop before the last element
        for k in range(i + 1, size):  # Ensure k is always greater than i
            bss_values.append(compute_intervariance(i, k))

    # Convert BSS values to a NumPy array
    bss_array = np.array(bss_values)

    # Find indices of maximum BSS
    max_bss_index = np.argmax(bss_array)
    
    # Convert the flat index back to i and k values
    i_max = max_bss_index // (size - 1)
    k_max = max_bss_index % (size - 1)

    # Return corresponding means for the threshold
    return means[i_max], means[k_max]

In [ ]:
def extract_shoreline(coll, datelist):
    shorelines = []
    smoothed_vectors = []

    image_list = coll.toList(coll.size())  # Convert ImageCollection to List
    for i, date in enumerate(datelist):
        im = ee.Image(image_list.get(i))
        hist = get_histogram(im, 10, 150, 'NDWI')
    
        t1, t2 = otsu_trimodal_from_histogram(hist)
        if t1 < t2:
            ndwi_thresh = [t1, t2]
        else:
            ndwi_thresh = [t2, t1]
        
        water_mask = im.lt(ee.Number(ndwi_thresh[0]))
        land_mask = im.gt(ee.Number(ndwi_thresh[1]))
        water_land_mask = water_mask.Not().add(land_mask).divide(2)
        shores = water_land_mask.updateMask(water_land_mask.gt(0.0))
    
        binary_mask = im.updateMask(im.gte(0.0)).mask()
        dilated_mask = binary_mask.focal_max(radius=10, units = 'meters')
        eroded_mask = binary_mask.focal_min(radius=10, units = 'meters')
        border_mask = dilated_mask.subtract(eroded_mask)
        shoreline = shores.updateMask(border_mask) ###should be shoreline

        shorelines.append(shoreline)  # shoreline
        
    return ee.ImageCollection(shorelines)

def extract_vegline(coll, datelist):
    veglines = []
    
    image_list = coll.toList(coll.size())  # Convert ImageCollection to List
    for i, date in enumerate(datelist):
        im = ee.Image(image_list.get(i))
        hist = get_histogram(im, 10, 150, 'NDVI')
    
        t1, t2 = otsu_trimodal_from_histogram(hist)
        if t1 < t2:
            ndvi_thresh = [t1, t2]
        else:
            ndvi_thresh = [t2, t1]
        
        fv_mask = im.lt(ee.Number(ndvi_thresh[0]))
        dv_mask = im.gt(ee.Number(ndvi_thresh[1]))
        veg_land_mask = fv_mask.Not().add(dv_mask).divide(2)
        veg = veg_land_mask.updateMask(veg_land_mask.gt(0.0))
    
        binary_mask = im.updateMask(im.gte(0.5)).mask()
        dilated_mask = binary_mask.focal_max(radius=10, units = 'meters')
        eroded_mask = binary_mask.focal_min(radius=10, units = 'meters')
        border_mask = dilated_mask.subtract(eroded_mask)
        vegline = veg.updateMask(border_mask) ###should be shoreline

        veglines.append(vegline)

    return ee.ImageCollection(veglines)


In [ ]:
# Function to calculate statistics for NDVI and NDMI over AOIs
def extract_statistics(image, index_name):
    # Reduce the image over the AOIs using mean, max, and min reducers
    stats = image.reduceRegions(
        collection=aois,
        reducer=ee.Reducer.minMax().combine(
            reducer2=ee.Reducer.percentile([25, 50, 75]).combine(
                reducer2=ee.Reducer.mean().combine(
                    reducer2=ee.Reducer.stdDev(),
                    sharedInputs=True
                ),
                sharedInputs=True
            ),
            sharedInputs=True
        ),
        scale=10
    )
    
    # Add the date of the image as a property to each feature in the collection
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    stats = stats.map(lambda f: f.set('date', date))
    
    return stats

# my function needs im_coll, index_name, and aoi

def extract_stats_from_aoi(image_collection, index_name, aoi):
    stats_ims = image_collection.map(lambda img: img.clip(aoi))
    stats_collection = stats_ims.map(lambda img: extract_statistics(img, index_name)).flatten()
    statslist = stats_collection.getInfo()['features']

    data=[]
    for feature in statslist:
        properties = feature['properties']
        data_dict = {}
        for key in properties:
            data_dict[f'{key}'] = properties[key]
        data.append(data_dict)

    df = pd.DataFrame(data)

    # Convert the date column to datetime
    df['date'] = pd.to_datetime(df['date'])

    # Sort the DataFrame by date
    df = df.sort_values(by='date')
    df.dropna(inplace=True)
    return df

In [ ]:
def plot_VI_ts(aoi, aoi_str, plottype):
    # Plotting the mean, standard deviation, and quartiles
    if plottype == 'together':
    
        plt.figure(figsize=(14, 7))


        for imcoll in [total_colls['s2_10m_ndvi'], total_colls['s2_10m_ndwi'], total_colls['s2_10m_msavi2']]:
            if imcoll == total_colls['s2_10m_ndvi']:
                df = extract_stats_from_aoi(imcoll, 'NDVI', aoi)
                plt.plot(df['date'], df['mean'], color='green', marker='o', linestyle='-', label='Mean NDVI')
                plt.fill_between(df['date'],
                                df['mean'] - df['stdDev'],
                                df['mean'] + df['stdDev'],
                                color='green', alpha=0.2, label='NDVI ± 1 Std Dev'
                            )

                # Plot NDVI Interquartile Range (IQR)
                plt.fill_between(df['date'],
                                df['p25'],
                                df['p75'],
                                color='green', alpha=0.5, linestyle='-', label='NDVI IQR (25th to 75th Percentile)'
                            )

            elif imcoll == total_colls['s2_10m_ndwi']:
            # Plot NDVI Mean with ±1 Std Dev Shading
                df = extract_stats_from_aoi(imcoll, 'NDWI', aoi)
                plt.plot(df['date'], df['mean'], color='blue', marker='o', linestyle='-', label='Mean NDWI')
                plt.fill_between(df['date'],
                                    df['mean'] - df['stdDev'],
                                    df['mean'] + df['stdDev'],
                                    color='blue', alpha=0.2, label='NDVI ± 1 Std Dev'
                                )

                # Plot NDVI Interquartile Range (IQR)
                plt.fill_between(df['date'],
                                    df['p25'],
                                    df['p75'],
                                    color='blue', alpha=0.5, linestyle='-', label='NDWI IQR (25th to 75th Percentile)'
                                )
                
            elif imcoll == total_colls['s2_10m_msavi2']:
                df = extract_stats_from_aoi(imcoll, 'MSAVI2', aoi)
                plt.plot(df['date'], df['mean'], color='red', marker='o', linestyle='-', label='Mean MSAVI2')
                plt.fill_between(df['date'],
                                df['mean'] - df['stdDev'],
                                df['mean'] + df['stdDev'],
                                color='red', alpha=0.2, label='MSAVI2 ± 1 Std Dev'
                            )

                # Plot MSAVI2 Interquartile Range (IQR)
                plt.fill_between(df['date'],
                                df['p25'],
                                df['p75'],
                                color='red', alpha=0.5, linestyle='-', label='MSAVI2 IQR (25th to 75th Percentile)'
                            )

        # Add vertical lines for Hurricane Laura and Hurricane Delta
        storm_dates = {
            'Hurricane Laura': pd.to_datetime('2020-08-27'),
            'Hurricane Delta': pd.to_datetime('2020-10-09'),
            'Hurricane Zeta': pd.to_datetime('2020-10-28'),
            'Hurricane Ida': pd.to_datetime('2021-08-29'),
            'Hurricane Beryl': pd.to_datetime('2024-07-24'),
            'Hurricane Francine': pd.to_datetime('2024-09-11')
        }

        for storm, date in storm_dates.items():
            if storm[0] == 'H':
                plt.axvline(x=date, color='purple', linestyle='--', linewidth=2)
                
        # Customizing the plot
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'NDVI, NDWI, and MSAVI2 Change Over Time for {aoi_str}')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.grid(True)

        # Show plot
        plt.show()
    
    elif plottype == 'separate':
        
        plt.figure(figsize=(14,7))

        df = extract_stats_from_aoi(total_colls['s2_10m_ndvi'], 'NDVI', aoi)
        plt.plot(df['date'], df['mean'], color='green', marker='o', linestyle='-', label='Mean NDVI')
        plt.fill_between(df['date'],
                        df['mean'] - df['stdDev'],
                        df['mean'] + df['stdDev'],
                        color='green', alpha=0.2, label='NDVI ± 1 Std Dev'
                    )

        # Plot NDVI Interquartile Range (IQR)
        plt.fill_between(df['date'],
                        df['p25'],
                        df['p75'],
                        color='green', alpha=0.5, linestyle='-', label='NDVI IQR (25th to 75th Percentile)'
                    )
        
        # Add vertical lines for Hurricane Laura and Hurricane Delta
        storm_dates = {
            'Hurricane Laura': pd.to_datetime('2020-08-27'),
            'Hurricane Delta': pd.to_datetime('2020-10-09'),
            'Hurricane Zeta': pd.to_datetime('2020-10-28'),
            'Hurricane Ida': pd.to_datetime('2021-08-29'),
            'Hurricane Beryl': pd.to_datetime('2024-07-24'),
            'Hurricane Francine': pd.to_datetime('2024-09-11')
        }

        for storm, date in storm_dates.items():
            if storm[0] == 'H':
                plt.axvline(x=date, color='purple', linestyle='--', linewidth=2)

        # Customizing the plot
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'NDVI Change Over Time for {aoi_str}')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.grid(True)

        # Show plot
        plt.show()

        plt.figure(figsize=(14,7))
        df = extract_stats_from_aoi(total_colls['s2_10m_ndwi'], 'NDWI', aoi)
        plt.plot(df['date'], df['mean'], color='blue', marker='o', linestyle='-', label='Mean NDWI')
        plt.fill_between(df['date'],
                        df['mean'] - df['stdDev'],
                        df['mean'] + df['stdDev'],
                        color='blue', alpha=0.2, label='NDWI ± 1 Std Dev'
                    )

        # Plot NDWI Interquartile Range (IQR)
        plt.fill_between(df['date'],
                        df['p25'],
                        df['p75'],
                        color='blue', alpha=0.5, linestyle='-', label='NDWI IQR (25th to 75th Percentile)'
                    )
        
        # Add vertical lines for Hurricane Laura and Hurricane Delta
        storm_dates = {
            'Hurricane Laura': pd.to_datetime('2020-08-27'),
            'Hurricane Delta': pd.to_datetime('2020-10-09'),
            'Hurricane Zeta': pd.to_datetime('2020-10-28'),
            'Hurricane Ida': pd.to_datetime('2021-08-29'),
            'Hurricane Beryl': pd.to_datetime('2024-07-24'),
            'Hurricane Francine': pd.to_datetime('2024-09-11')
        }

        for storm, date in storm_dates.items():
            if storm[0] == 'H':
                plt.axvline(x=date, color='purple', linestyle='--', linewidth=2)

        # Customizing the plot
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'NDWI Change Over Time for {aoi_str}')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.grid(True)

        # Show plot
        plt.show()

        plt.figure(figsize=(14,7))

        df = extract_stats_from_aoi(total_colls['s2_10m_msavi2'], 'MSAVI2', aoi)
        plt.plot(df['date'], df['mean'], color='red', marker='o', linestyle='-', label='Mean MSAVI2')
        plt.fill_between(df['date'],
                        df['mean'] - df['stdDev'],
                        df['mean'] + df['stdDev'],
                        color='red', alpha=0.2, label='MSAVI2 ± 1 Std Dev'
                    )

        # Plot NDVI Interquartile Range (IQR)
        plt.fill_between(df['date'],
                        df['p25'],
                        df['p75'],
                        color='red', alpha=0.5, linestyle='-', label='MSAVI2 IQR (25th to 75th Percentile)'
                    )
        
        # Add vertical lines for Hurricane Laura and Hurricane Delta
        storm_dates = {
            'Hurricane Laura': pd.to_datetime('2020-08-27'),
            'Hurricane Delta': pd.to_datetime('2020-10-09'),
            'Hurricane Zeta': pd.to_datetime('2020-10-28'),
            'Hurricane Ida': pd.to_datetime('2021-08-29'),
            'Hurricane Beryl': pd.to_datetime('2024-07-24'),
            'Hurricane Francine': pd.to_datetime('2024-09-11')
        }

        for storm, date in storm_dates.items():
            if storm[0] == 'H':
                plt.axvline(x=date, color='purple', linestyle='--', linewidth=2)

        # Customizing the plot
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'MSAVI2 Change Over Time for {aoi_str}')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.grid(True)

        # Show plot
        plt.show()

In [ ]:
def plot_GRD_ts(aoi, aoi_str, plottype):
    # Plotting the mean, standard deviation, and quartiles
    if plottype == 'together':
        plt.figure(figsize=(14, 7))


        for imcoll in [total_colls['s1_VV'], total_colls['s1_VH']]:
            if imcoll == total_colls['s1_VV']:
                df = extract_stats_from_aoi(imcoll, 'VV', aoi)
                plt.plot(df['date'], df['mean'], color='green', marker='o', linestyle='-', label='Mean VV Backscatter')
                plt.fill_between(df['date'],
                                df['mean'] - df['stdDev'],
                                df['mean'] + df['stdDev'],
                                color='green', alpha=0.2, label='VV Backscatter ± 1 Std Dev'
                            )

                # Plot VV Interquartile Range (IQR)
                plt.fill_between(df['date'],
                                df['p25'],
                                df['p75'],
                                color='green', alpha=0.5, linestyle='-', label='VV Backascatter IQR (25th to 75th Percentile)'
                            )

            elif imcoll == total_colls['s1_VH']:
            # Plot VH Mean with ±1 Std Dev Shading
                df = extract_stats_from_aoi(imcoll, 'VH', aoi)
                plt.plot(df['date'], df['mean'], color='blue', marker='o', linestyle='-', label='Mean VH Backscatter')
                plt.fill_between(df['date'],
                                    df['mean'] - df['stdDev'],
                                    df['mean'] + df['stdDev'],
                                    color='blue', alpha=0.2, label='VH Backscatter ± 1 Std Dev'
                                )

                # Plot VH Interquartile Range (IQR)
                plt.fill_between(df['date'],
                                    df['p25'],
                                    df['p75'],
                                    color='blue', alpha=0.5, linestyle='-', label='VH Backscatter IQR (25th to 75th Percentile)'
                                )


        # Add vertical lines for Hurricane Laura and Hurricane Delta
        storm_dates = {
            'Hurricane Laura': pd.to_datetime('2020-08-27'),
            'Hurricane Delta': pd.to_datetime('2020-10-09'),
            'Hurricane Zeta': pd.to_datetime('2020-10-28'),
            'Hurricane Ida': pd.to_datetime('2021-08-29'),
            'Hurricane Beryl': pd.to_datetime('2024-07-24'),
            'Hurricane Francine': pd.to_datetime('2024-09-11')
        }

        for storm, date in storm_dates.items():
            if storm[0] == 'H':
                plt.axvline(x=date, color='purple', linestyle='--', linewidth=2)

        
        # Customizing the plot
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'Sentinel-1 Backscatter Amplitude Change Over Time for {aoi_str}')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.grid(True)

        # Show plot
        plt.show()

    elif plottype == 'separate':
        df = extract_stats_from_aoi(total_colls['s1_VV'], 'VV', aoi)
        plt.plot(df['date'], df['mean'], color='green', marker='o', linestyle='-', label='Mean VV Backscatter')
        plt.fill_between(df['date'],
                        df['mean'] - df['stdDev'],
                        df['mean'] + df['stdDev'],
                        color='green', alpha=0.2, label='VV Backscatter ± 1 Std Dev'
                    )

        # Plot VV Interquartile Range (IQR)
        plt.fill_between(df['date'],
                        df['p25'],
                        df['p75'],
                        color='green', alpha=0.5, linestyle='-', label='VV Backascatter IQR (25th to 75th Percentile)'
                    )
        
        # Add vertical lines for Hurricane Laura and Hurricane Delta
        storm_dates = {
            'Hurricane Laura': pd.to_datetime('2020-08-27'),
            'Hurricane Delta': pd.to_datetime('2020-10-09'),
            'Hurricane Zeta': pd.to_datetime('2020-10-28'),
            'Hurricane Ida': pd.to_datetime('2021-08-29'),
            'Hurricane Beryl': pd.to_datetime('2024-07-24'),
            'Hurricane Francine': pd.to_datetime('2024-09-11')
        }

        for storm, date in storm_dates.items():
            if storm[0] == 'H':
                plt.axvline(x=date, color='purple', linestyle='--', linewidth=2)

        # Customizing the plot
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'Sentinel-1 VV Backscatter Amplitude Change Over Time for {aoi_str}')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.grid(True)

        # Show plot
        plt.show()


        df = extract_stats_from_aoi(total_colls['s1_VH'], 'VH', aoi)
        plt.plot(df['date'], df['mean'], color='blue', marker='o', linestyle='-', label='Mean VH Backscatter')
        plt.fill_between(df['date'],
                        df['mean'] - df['stdDev'],
                        df['mean'] + df['stdDev'],
                        color='blue', alpha=0.2, label='VH Backscatter ± 1 Std Dev'
                    )

        # Plot VV Interquartile Range (IQR)
        plt.fill_between(df['date'],
                        df['p25'],
                        df['p75'],
                        color='blue', alpha=0.5, linestyle='-', label='VH Backascatter IQR (25th to 75th Percentile)'
                    )
        

        # Add vertical lines for Hurricane Laura and Hurricane Delta
        storm_dates = {
            'Hurricane Laura': pd.to_datetime('2020-08-27'),
            'Hurricane Delta': pd.to_datetime('2020-10-09'),
            'Hurricane Zeta': pd.to_datetime('2020-10-28'),
            'Hurricane Ida': pd.to_datetime('2021-08-29'),
            'Hurricane Beryl': pd.to_datetime('2024-07-24'),
            'Hurricane Francine': pd.to_datetime('2024-09-11')
        }

        for storm, date in storm_dates.items():
            if storm[0] == 'H':
                plt.axvline(x=date, color='purple', linestyle='--', linewidth=2)

        # Customizing the plot
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'Sentinel-1 VH Backscatter Amplitude Change Over Time for {aoi_str}')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.grid(True)

        # Show plot
        plt.show()

# Get area of interest

In [ ]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='HYBRID')
## Add drawing tools
m.add_draw_control()
## Display the map
display(m)

In [ ]:
## Get the drawn features
draw_features = m.draw_features[0]
## Establish ee.Polygon from drawn area of interest to collect imagery
aoi = ee.Geometry.Polygon(draw_features.getInfo()['geometry']['coordinates'][0])

# Get Imagery

In [ ]:
start_date = '2018-01-01' ## start date of search window
end_date = '2024-10-03' ## end date of search window
s2_cloud_cov = 10 ## percentage of clouds in sentinel-2 multispectral imagery, less means you see more surface
orbit = 'ASCENDING' ## orbit for imagery

collections = {}
for i in range(int(start_date[:4]), int(end_date[:4])):
    collections[f's1_VV_{i}_{i+1}'], collections[f's1_VH_{i}_{i+1}'], collections[f's2_10m_{i}_{i+1}'], collections[f's2_10m_ndvi_{i}_{i+1}'], collections[f's2_10m_ndwi_{i}_{i+1}'], collections[f's2_10m_msavi2_{i}_{i+1}']  = get_sentinel_imagery(aoi, f'{i}{start_date[4:]}', f'{i+1}{end_date[4:]}', s2_cloud_cov, orbit)

In [ ]:
total_colls = {}
total_colls['s1_VV'], total_colls['s1_VH'], total_colls['s2_10m'], total_colls['s2_10m_ndvi'], total_colls['s2_10m_ndwi'], total_colls['s2_10m_msavi2']= get_sentinel_imagery(aoi, start_date, end_date, s2_cloud_cov, orbit)

In [ ]:
colls = []
for i, coll in enumerate(collections):
    if collections[coll].size().getInfo() != 0: # removes image collections that are empty
        colls.append(coll)

for i, coll in enumerate(colls):
    print(f'{i}: {coll}')

# Visualize Distribution of Spectral Indices Values for Select Images

In [ ]:
# no mask

# all I need for segmenting images into coastlines would be
# Need to make this applicable for each image in image collection
# it will read in a specified band of a passed image collection, then segment it using Otsu's method
# It should create a new image collection of 
#   1) Water Bodies, 2) Land Bodies, 3) Shorelines
# those can then be used to measure horizontal changes in shoreline over time

# Get the first image in the collection (or map over the collection if needed)
s2_date_list = collections[colls[4]].map(get_date).aggregate_array('date').getInfo()
ndwi_image = collections[colls[4]].first()
ndwi_image = ndwi_image.updateMask(ndwi_image.lt(0.0))

ndwi_date = s2_date_list[0]
ndwi_histogram = get_histogram(ndwi_image, 10, 150, 'NDWI')

# Extract the histogram values for plotting
ndwi_values = ndwi_histogram['bucketMeans']
ndwi_counts = ndwi_histogram['histogram']


# second date
# Get the first image in the collection (or map over the collection if needed)
s2_date_list2 = collections[colls[22]].map(get_date).aggregate_array('date').getInfo()
ndwi_image2 = collections[colls[22]].first()
ndwi_image2 = ndwi_image2.updateMask(ndwi_image2.lt(0.0))


ndwi_date2 = s2_date_list2[0]
ndwi_histogram2 = get_histogram(ndwi_image2, 10, 150, 'NDWI')

# Extract the histogram values for plotting
ndwi_values2 = ndwi_histogram2['bucketMeans']
ndwi_counts2 = ndwi_histogram2['histogram']

# Plot the histogram using Matplotlib
plt.figure(figsize=(10, 6))
plt.bar(ndwi_values, ndwi_counts, width=0.02, color='blue', alpha=0.7, label = ndwi_date)
plt.bar(ndwi_values2, ndwi_counts2, width=0.02, color='red', alpha=0.4, label = ndwi_date2)
plt.title(f"NDWI Histogram")
plt.xlabel("NDWI Values")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.show()

edge_i, edge_k = otsu_trimodal_from_histogram(ndwi_histogram)
print(f'NDWI Thershold 1 {ndwi_date}= {edge_i}')
print(f'NDWI Thershold 2 {ndwi_date}= {edge_k}')

if edge_i < edge_k:
    ndwi_thresh = [edge_i, edge_k]
else:
    ndwi_thresh = [edge_k, edge_i]

edge_i2, edge_k2 = otsu_trimodal_from_histogram(ndwi_histogram2)
print(f'NDWI Thershold 1 {ndwi_date2}= {edge_i2}')
print(f'NDWI Thershold 2 {ndwi_date2}= {edge_k2}')

if edge_i2 < edge_k2:
    ndwi_thresh2 = [edge_i2, edge_k2]
else:
    ndwi_thresh2 = [edge_k2, edge_i2]

In [ ]:
# no mask

# all I need for segmenting images into coastlines would be
# Need to make this applicable for each image in image collection
# it will read in a specified band of a passed image collection, then segment it using Otsu's method
# It should create a new image collection of 
#   1) Water Bodies, 0) Land Bodies, 3) Shorelines
# those can then be used to measure horizontal changes in shoreline over time

# Get the first image in the collection (or map over the collection if needed)
s2_date_list = collections[colls[3]].map(get_date).aggregate_array('date').getInfo()
ndvi_image = collections[colls[3]].first()
ndvi_image = ndvi_image.updateMask(ndvi_image.gte(0.0))     ### mask to remove the water and low vegetation areas

ndvi_date = s2_date_list[0]
ndvi_histogram = get_histogram(ndvi_image, 10, 150, 'NDVI')

# Extract the histogram values for plotting
ndvi_values = ndvi_histogram['bucketMeans']
ndvi_counts = ndvi_histogram['histogram']


# second date
# Get the first image in the collection (or map over the collection if needed)
s1_date_list2 = collections[colls[21]].map(get_date).aggregate_array('date').getInfo()
ndvi_image2 = collections[colls[21]].first()
ndvi_image2 = ndvi_image2.updateMask(ndvi_image2.gte(0.0))

ndvi_date2 = s1_date_list2[0]
ndvi_histogram2 = get_histogram(ndvi_image2, 10, 150, 'NDVI')

# Extract the histogram values for plotting
ndvi_values2 = ndvi_histogram2['bucketMeans']
ndvi_counts2 = ndvi_histogram2['histogram']

# Plot the histogram using Matplotlib
plt.figure(figsize=(10, 6))
plt.bar(ndvi_values, ndvi_counts, width=0.02, color='blue', alpha=0.7, label = ndvi_date)
plt.bar(ndvi_values2, ndvi_counts2, width=0.02, color='red', alpha=0.4, label = ndvi_date2)
plt.title(f"NDVI Histogram")
plt.xlabel("NDVI Values")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.show()

edge_i, edge_k = otsu_trimodal_from_histogram(ndvi_histogram)
if edge_i < edge_k:
    ndvi_thresh = [edge_i, edge_k]
else:
    ndvi_thresh = [edge_k, edge_i]

edge_i2, edge_k2 = otsu_trimodal_from_histogram(ndvi_histogram2)
if edge_i2 < edge_k2:
    ndvi_thresh2 = [edge_i2, edge_k2]
else:
    ndvi_thresh2 = [edge_k2, edge_i2]

print(f'NDVI Thersholds for {ndvi_date} = {ndvi_thresh}')
print(f'NDVI Thersholds for {ndvi_date2} = {ndvi_thresh2}')

In [ ]:
s2_date_list = collections[colls[5]].map(get_date).aggregate_array('date').getInfo()
msavi2_image = collections[colls[5]].first()
msavi2_image = msavi2_image.updateMask(msavi2_image.gte(0.0))     ### mask to remove the water and low vegetation areas

msavi2_date = s2_date_list[0]
msavi2_histogram = get_histogram(msavi2_image, 10, 150, 'MSAVI2')

# Extract the histogram values for plotting
msavi2_values = msavi2_histogram['bucketMeans']
msavi2_counts = msavi2_histogram['histogram']


# second date
# Get the first image in the collection (or map over the collection if needed)
s2_date_list2 = collections[colls[23]].map(get_date).aggregate_array('date').getInfo()
msavi2_image2 = collections[colls[23]].first()
msavi2_image2 = msavi2_image2.updateMask(msavi2_image2.gte(0.0))

msavi2_date2 = s2_date_list2[0]
msavi2_histogram2 = get_histogram(msavi2_image2, 10, 150, 'MSAVI2')

# Extract the histogram values for plotting
msavi2_values2 = msavi2_histogram2['bucketMeans']
msavi2_counts2 = msavi2_histogram2['histogram']

# Plot the histogram using Matplotlib
plt.figure(figsize=(10, 6))
plt.bar(msavi2_values, msavi2_counts, width=0.02, color='blue', alpha=0.7, label = msavi2_date)
plt.bar(msavi2_values2, msavi2_counts2, width=0.02, color='red', alpha=0.4, label = msavi2_date2)
plt.title(f"MSAVI2 Histogram")
plt.xlabel("MSAVI2 Values")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.show()

edge_i, edge_k = otsu_trimodal_from_histogram(msavi2_histogram)
if edge_i < edge_k:
    msavi2_thresh = [edge_i, edge_k]
else:
    msavi2_thresh = [edge_k, edge_i]

edge_i2, edge_k2 = otsu_trimodal_from_histogram(msavi2_histogram2)
if edge_i2 < edge_k2:
    msavi2_thresh2 = [edge_i2, edge_k2]
else:
    msavi2_thresh2 = [edge_k2, edge_i2]

print(f'MSAVI2 Thersholds for {msavi2_date} = {msavi2_thresh}')
print(f'MSAVI2 Thersholds for {msavi2_date2} = {msavi2_thresh2}')

# Shoreline and Veg line detection based on Otsu Trimodal Thresholds

Get info of specific areas of information
- can be specific areas where change has occurred, in my application probably sediment enrichment
- (working) get a time series of NDVI and NDWI for the area delineated by the drawn aois and shoreline

In [ ]:
s2_date_list = collections[colls[4]].map(get_date).aggregate_array('date').getInfo()
s2_date = s2_date_list[0]

## Create a map centered at a specific location
m = geemap.Map()
m.centerObject(aoi, 12)

# Add the original rgb image for context
m.addLayer(collections[colls[2]].first(), {'min': 0, 'max': 2000, 'bands': ['B4', 'B3', 'B2']}, f'{s2_date} RGB')


#draw the specific aois you want to get time series info from
# for now needs to be an area
# later will add stuff for transects
m.add_draw_control()
m.addLayerControl()
display(m)


In [ ]:
# want to add another 
## Get the drawn features
drawn_features = m.draw_features

aois = ee.FeatureCollection(drawn_features)

sites = {}
for i, site in enumerate(drawn_features):
    sites[f'Site {i}'] = ee.Geometry.Polygon(site.getInfo()['geometry']['coordinates'][0])

In [ ]:
# s2_date_list = total_colls['s2_10m_ndwi'].map(get_date).aggregate_array('date').getInfo()
# shoreline_collection = extract_shoreline(coll = total_colls['s2_10m_ndwi'] , datelist =s2_date_list)
# vegline_collection = extract_vegline(coll = total_colls['s2_10m_ndvi'], datelist = s2_date_list )

# clipped_shores = shoreline_collection.map(lambda img: img.clip(aois))
# clipped_veg = vegline_collection.map(lambda img: img.clip(aois))
# clipped_rgb = total_colls['s2_10m'].map(lambda img: img.clip(aois))

# clipped_VV = total_colls['s1_VV'].map(lambda img: img.clip(aois))
# clipped_VH = total_colls['s1_VH'].map(lambda img: img.clip(aois))

In [ ]:
s2_date_list = collections[colls[20]].map(get_date).aggregate_array('date').getInfo()
shoreline_collection = extract_shoreline(coll = collections[colls[22]] , datelist =s2_date_list)
vegline_collection = extract_vegline(coll = collections[colls[21]], datelist = s2_date_list )

clipped_shores = shoreline_collection.map(lambda img: img.clip(aois))
clipped_veg = vegline_collection.map(lambda img: img.clip(aois))
clipped_rgb = collections[colls[20]].map(lambda img: img.clip(aois))

clipped_VV = collections[colls[18]].map(lambda img: img.clip(aois))
clipped_VH = collections[colls[19]].map(lambda img: img.clip(aois))
clipped_ndvi = collections[colls[21]].map(lambda img: img.clip(aois))
clipped_ndwi = collections[colls[22]].map(lambda img: img.clip(aois))
clipped_msavi2 = collections[colls[23]].map(lambda img: img.clip(aois))

In [ ]:
map_ts = geemap.Map()
map_ts.centerObject(aoi, 12)

rgb_images = clipped_rgb.toList(clipped_rgb.size())
for i in range(clipped_rgb.size().getInfo()//10):
    image = ee.Image(rgb_images.get(-i))
    add_rgb_to_map(image, map_ts, clipped_rgb)

    # VV_image = ee.Image((clipped_VV.toList(clipped_VV.size())).get(-i))
    # add_sar_to_map(VV_image, map_ts, 'VV')

    # VH_image = ee.Image((clipped_VH.toList(clipped_VH.size())).get(-i))
    # add_sar_to_map(VH_image, map_ts, 'VH')

    ndvi_image = ee.Image((clipped_ndvi.toList(clipped_ndvi.size())).get(-i))
    add_ind_to_map(ndvi_image, map_ts, 'NDVI')

    ndwi_image = ee.Image((clipped_ndwi.toList(clipped_ndwi.size())).get(-i))
    add_ind_to_map(ndwi_image, map_ts, 'NDWI')

    msavi2_image = ee.Image((clipped_msavi2.toList(clipped_msavi2.size())).get(-i))
    add_ind_to_map(msavi2_image, map_ts, 'MSAVI2')
    # shore_im = ee.Image(clipped_shores.toList(clipped_shores.size()).get(-i))
    # map_ts.addLayer(shore_im, {}, f'{s2_date_list[i]} shore')

    # veg_im = ee.Image(clipped_veg.toList(clipped_veg.size()).get(-i))
    # map_ts.addLayer(veg_im, {}, f'{s2_date_list[i]} veg')



# map_ts.add_draw_control()
# Display the map
map_ts.addLayerControl()

## Display the map
display(map_ts)

# Plot time series for the sites you identified

In [ ]:
for key in sites:
    print(key)

In [ ]:
## Time series plot to show changes in NDVI and NDWI
# plot_VI_ts(sites['Site 2'], 'Unit 1A', 'separate')
# plot_VI_ts(sites['Site 1'], 'Unit 1B', 'separate')
plot_VI_ts(sites['Site 1'], 'Unit 1D', 'separate') # site finished in Sept 2019
plot_VI_ts(sites['Site 0'], 'Unit 1E', 'separate') # site finished in Sept 2019
plot_VI_ts(sites['Site 2'], '2023 Site', 'separate') #Site Created in late 2023

In [ ]:
## Time series plot to show changes in Sentinel-1 GRD

# plot_GRD_ts(sites['Site 4'], 'Unit 1A', 'together')
# plot_GRD_ts(sites['Site 3'], 'Unit 1B', 'together')
plot_GRD_ts(sites['Site 1'], 'Unit 1D', 'together') # Site finished in Sept 2019
plot_GRD_ts(sites['Site 0'], 'Unit 1E', 'together') # Site finished in Sept 2019
plot_GRD_ts(sites['Site 2'], '2023 Site', 'together') # Site created in Late 2023

In [ ]:
def plot_VI_ts_combined(aoi, aoi_str, plottype):
    # Plotting the mean, standard deviation, and quartiles
    if plottype == 'together':
    
        plt.figure(figsize=(14, 7))

        for imcoll in [total_colls['s2_10m_ndvi'], total_colls['s2_10m_ndwi']]:
            if imcoll == total_colls['s2_10m_ndvi']:
                df_msavi2 = extract_stats_from_aoi(total_colls['s2_10m_msavi2'], 'MSAVI2', aoi)
                df_ndvi = extract_stats_from_aoi(imcoll, 'NDVI', aoi)

                # Replace NDVI/MSAVI2 values with NDVI where MSAVI2 > 0.6
                df_combined = df_msavi2.copy()
                condition = df_msavi2['mean'] > 0.6
                df_combined.loc[condition, 'mean'] = df_ndvi.loc[condition, 'mean']
                df_combined.loc[condition, 'stdDev'] = df_ndvi.loc[condition, 'stdDev']
                df_combined.loc[condition, 'p25'] = df_ndvi.loc[condition, 'p25']
                df_combined.loc[condition, 'p75'] = df_ndvi.loc[condition, 'p75']

                plt.plot(df_combined['date'], df_combined['mean'], color='green', marker='o', linestyle='-', label='Mean NDVI/MSAVI2 (with NDVI substitution)')
                plt.fill_between(df_combined['date'],
                                df_combined['mean'] - df_combined['stdDev'],
                                df_combined['mean'] + df_combined['stdDev'],
                                color='green', alpha=0.2, label='NDVI/MSAVI2 ± 1 Std Dev (with NDVI substitution)'
                            )

                # Plot NDVI/MSAVI2 Interquartile Range (IQR)
                plt.fill_between(df_combined['date'],
                                df_combined['p25'],
                                df_combined['p75'],
                                color='green', alpha=0.5, linestyle='-', label='NDVI/MSAVI2 IQR (with NDVI substitution)'
                            )

            elif imcoll == total_colls['s2_10m_ndwi']:
                df_ndwi = extract_stats_from_aoi(imcoll, 'NDWI', aoi)
                plt.plot(df_ndwi['date'], df_ndwi['mean'], color='blue', marker='o', linestyle='-', label='Mean NDWI')
                plt.fill_between(df_ndwi['date'],
                                    df_ndwi['mean'] - df_ndwi['stdDev'],
                                    df_ndwi['mean'] + df_ndwi['stdDev'],
                                    color='blue', alpha=0.2, label='NDWI ± 1 Std Dev'
                                )

                # Plot NDWI Interquartile Range (IQR)
                plt.fill_between(df_ndwi['date'],
                                    df_ndwi['p25'],
                                    df_ndwi['p75'],
                                    color='blue', alpha=0.5, linestyle='-', label='NDWI IQR (25th to 75th Percentile)'
                                )
        


        # Add vertical lines for Hurricane Laura and Hurricane Delta
        storm_dates = {
            'Hurricane Laura': pd.to_datetime('2020-08-27'),
            'Hurricane Delta': pd.to_datetime('2020-10-09'),
            'Hurricane Zeta': pd.to_datetime('2020-10-28'),
            'Hurricane Ida': pd.to_datetime('2021-08-29'),
            'Hurricane Beryl': pd.to_datetime('2024-07-24'),
            'Hurricane Francine': pd.to_datetime('2024-09-11')
        }

        for storm, date in storm_dates.items():
            if storm[0] == 'H':
                plt.axvline(x=date, color='purple', linestyle='--', linewidth=2)

        # Customizing the plot
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'NDVI/MSAVI2 and NDWI Change Over Time for {aoi_str}')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.grid(True)

        # Show plot
        plt.show()
        
    elif plottype == 'separate':
        
        plt.figure(figsize=(14,7))

        df_ndvi = extract_stats_from_aoi(total_colls['s2_10m_ndvi'], 'NDVI', aoi)
        df_msavi2 = extract_stats_from_aoi(total_colls['s2_10m_msavi2'], 'MSAVI2', aoi)

        # Replace NDVI/MSAVI2 values with NDVI where MSAVI2 > 0.6
        df_combined = df_msavi2.copy()
        condition = df_msavi2['mean'] > 0.6
        df_combined.loc[condition, 'mean'] = df_ndvi.loc[condition, 'mean']
        df_combined.loc[condition, 'stdDev'] = df_ndvi.loc[condition, 'stdDev']
        df_combined.loc[condition, 'p25'] = df_ndvi.loc[condition, 'p25']
        df_combined.loc[condition, 'p75'] = df_ndvi.loc[condition, 'p75']

        plt.plot(df_combined['date'], df_combined['mean'], color='green', marker='o', linestyle='-', label='Mean NDVI/MSAVI2 (with NDVI substitution)')
        plt.fill_between(df_combined['date'],
                        df_combined['mean'] - df_combined['stdDev'],
                        df_combined['mean'] + df_combined['stdDev'],
                        color='green', alpha=0.2, label='NDVI/MSAVI2 ± 1 Std Dev (with NDVI substitution)'
                    )

        # Plot NDVI/MSAVI2 Interquartile Range (IQR)
        plt.fill_between(df_combined['date'],
                        df_combined['p25'],
                        df_combined['p75'],
                        color='green', alpha=0.5, linestyle='-', label='NDVI/MSAVI2 IQR (with NDVI substitution)'
                    )
        
        # Add vertical lines for Hurricane Laura and Hurricane Delta
        storm_dates = {
            'Hurricane Laura': pd.to_datetime('2020-08-27'),
            'Hurricane Delta': pd.to_datetime('2020-10-09'),
            'Hurricane Zeta': pd.to_datetime('2020-10-28'),
            'Hurricane Ida': pd.to_datetime('2021-08-29'),
            'Hurricane Beryl': pd.to_datetime('2024-07-24'),
            'Hurricane Francine': pd.to_datetime('2024-09-11')
        }

        for storm, date in storm_dates.items():
            if storm[0] == 'H':
                plt.axvline(x=date, color='purple', linestyle='--', linewidth=2)

        # Customizing the plot
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'Vegetation Change Over Time for {aoi_str}')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.grid(True)

        # Show plot
        plt.show()

        plt.figure(figsize=(14,7))

        df = extract_stats_from_aoi(total_colls['s2_10m_ndwi'], 'NDWI', aoi)
        plt.plot(df['date'], df['mean'], color='blue', marker='o', linestyle='-', label='Mean NDWI')
        plt.fill_between(df['date'],
                        df['mean'] - df['stdDev'],
                        df['mean'] + df['stdDev'],
                        color='blue', alpha=0.2, label='NDWI ± 1 Std Dev'
                    )

        # Plot NDWI Interquartile Range (IQR)
        plt.fill_between(df['date'],
                        df['p25'],
                        df['p75'],
                        color='blue', alpha=0.5, linestyle='-', label='NDWI IQR (25th to 75th Percentile)'
                    )
        
        # Add vertical lines for Hurricane Laura and Hurricane Delta
        storm_dates = {
            'Hurricane Laura': pd.to_datetime('2020-08-27'),
            'Hurricane Delta': pd.to_datetime('2020-10-09'),
            'Hurricane Zeta': pd.to_datetime('2020-10-28'),
            'Hurricane Ida': pd.to_datetime('2021-08-29'),
            'Hurricane Beryl': pd.to_datetime('2024-07-24'),
            'Hurricane Francine': pd.to_datetime('2024-09-11')
        }

        for storm, date in storm_dates.items():
            if storm[0] == 'H':
                plt.axvline(x=date, color='purple', linestyle='--', linewidth=2)

        # Customizing the plot
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'NDWI Change Over Time for {aoi_str}')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.grid(True)

        # Show plot
        plt.show()

plot_VI_ts_combined(sites['Site 0'], 'Unit 1E', 'together')
plot_VI_ts_combined(sites['Site 1'], 'Unit 1D', 'together')
plot_VI_ts_combined(sites['Site 2'], '2023 Site', 'together')

- Mean NDVI/NDWI Above Q3: small proportion of higher values pulling the mean upwards—perhaps due to localized greening or a recent vegetation growth spurt affecting only part of the area.
- Mean NDVI/NDWI Below Q1: extreme low values are present, pulling the mean down—potentially due to vegetation loss, seasonal dry periods, or land cover change in part of the region.

# Testing
- Get spatial statisitics from drawn_features, plot them in a time-series to show upper-value, lower-value, and mean-value for NDWI and NDVI. Will show evolution of vegetation cover and surface moisture from Sentinel-2
- Vectorize the shoreline? Need to get the horizontal changes in the identified shorelines between subsequent changes
- Once able to spatially estimate shoreline changes, use those to estimate area changes for the marsh creation sites. (Once the area is estiamted, the volumetric changes can be estimated using closmure phase corrected InSAR)

In [ ]:
def extract_shoreline_test(coll, datelist):
    shorelines = []
    smoothed_vectors = []

    image_list = coll.toList(coll.size())  # Convert ImageCollection to List
    for i, date in enumerate(datelist):
        im = ee.Image(image_list.get(i))
        ndvi_im = ee.Image(collections[colls[15]].toList(collections[colls[15]].size()).get(i))
        im = im.updateMask(ndvi_im.gt(0.0))
        hist = get_histogram(im, 10, 150, 'NDWI')
    
        t1, t2 = otsu_trimodal_from_histogram(hist)
        if t1 < t2:
            ndwi_thresh = [t1, t2]
        else:
            ndwi_thresh = [t2, t1]
        
        water_mask = im.lt(ee.Number(ndwi_thresh[0]))
        land_mask = im.gt(ee.Number(ndwi_thresh[1]))
        combo_mask = water_mask.Not().add(land_mask).divide(2)
        shores = combo_mask.updateMask(combo_mask.gt(0.0))
    
        binary_mask = im.updateMask(im.gte(0.0)).mask()
        dilated_mask = binary_mask.focal_max(radius=10, units = 'meters')
        eroded_mask = binary_mask.focal_min(radius=10, units = 'meters')
        border_mask = dilated_mask.subtract(eroded_mask)
        shoreline = shores.updateMask(border_mask) ###should be shoreline

        shorelines.append(shoreline)  # shoreline
        
    return ee.ImageCollection(shorelines)

In [ ]:
s2_date_list = collections[colls[14]].map(get_date).aggregate_array('date').getInfo()
testcoll = extract_shoreline_test(coll = collections[colls[16]] , datelist =s2_date_list)

clipped_shores_test = testcoll.map(lambda img: img.clip(aois))
clipped_rgb = collections[colls[14]].map(lambda img: img.clip(aois))

clipped_VV = collections[colls[12]].map(lambda img: img.clip(aois))
clipped_VH = collections[colls[13]].map(lambda img: img.clip(aois))

In [ ]:
# cannyedge = ee.Algorithms.CannyEdgeDetector(clipped_shores.first(), threshold = 0.5, sigma =1)

vectors = clipped_shores_test.first().toInt().reduceToVectors(
# vectors = cannyedge.toInt().reduceToVectors(
    geometry = sites['Site 0'],
    geometryType='polygon', 
    reducer=ee.Reducer.countEvery(),
    scale=10  # Use an appropriate scale for your data
)

# Define a function that converts polygons to lines by using the .difference() method
def polygon_to_line(feature):
    geometry = feature.geometry()
    # Using .difference() to convert a polygon to a line by removing the interior
    line_geometry = geometry.difference(geometry.buffer(-1))
    return feature.setGeometry(line_geometry)

# Apply the function to convert polygons to lines
lines = vectors.map(polygon_to_line)

In [ ]:
maptest = geemap.Map()
maptest.centerObject(aoi, 12)

rgb_images = clipped_rgb.toList(clipped_rgb.size())
for i in range(clipped_rgb.size().getInfo()//4):
    image = ee.Image(rgb_images.get(-i))
    add_rgb_to_map(image, maptest, clipped_rgb)

    # sar_image = ee.Image((clipped_VV.toList(clipped_VV.size())).get(-i))
    # add_sar_to_map(sar_image, maptest, 'VV')

    shore_im = ee.Image(clipped_shores_test.toList(clipped_shores.size()).get(-i))
    maptest.addLayer(shore_im, {}, f'{s2_date_list[i]} shore')

    # veg_im = ee.Image(clipped_veg.toList(clipped_veg.size()).get(-i))
    # maptest.addLayer(veg_im, {}, f'{s2_date_list[i]} veg')


# maptest.addLayer(vectors, {}, f'test')
# maptest.addLayer(lines, {}, f'test')
# maptest.add_draw_control()
# Display the map
maptest.addLayerControl()

## Display the map
display(maptest)

In [ ]:
maptest = geemap.Map()
maptest.centerObject(aoi, 12)


# Visualize each image in the ImageCollection.
rgb_images = clipped_rgb.toList(clipped_rgb.size())
for i in range(clipped_rgb.size().getInfo()//5):
    image = ee.Image(rgb_images.get(i))
    add_rgb_to_map(image, maptest, clipped_rgb)
    
    shore_im = ee.Image(clipped_shores.toList(clipped_shores.size()).get(i))
    maptest.addLayer(shore_im, {'palette': 'black'}, f'{s2_date_list[i]} shore')

    veg_im = ee.Image(clipped_veg.toList(clipped_veg.size()).get(i))
    maptest.addLayer(veg_im, {'palette': 'red'}, f'{s2_date_list[i]} veg')

# Display the map.
maptest.addLayerControl(position = 'topright')
maptest

In [ ]:
# vectorize the shorelines so that I can clip the images by them
### how to do this????


# Export Shoreline information

In [ ]:
# Export the shoreline vector as GeoJSON for use in QGIS or GDAL
task = ee.batch.Export.table.toDrive(
    collection=vector_shoreline,
    description='Shoreline_Export',
    fileFormat='GeoJSON',  # GeoJSON is well-supported in QGIS and GDAL
    fileNamePrefix='shoreline'
)

# Start the export task
task.start()
print("Shoreline extraction task started, check Google Earth Engine tasks for progress.")

In [ ]:
Export.image.toDrive({
    'image': first_edge_image,
    'description': 'Canny_Edges_Example',
    'scale': 10,  # Adjust the scale according to the dataset resolution
    'region': aoi,
    'maxPixels': 1e8
})